In [1]:
import hipscat_import.pipeline as runner
from hipscat.pixel_math.hipscat_id import HIPSCAT_ID_COLUMN
from hipscat_import.catalog.arguments import ImportArguments

from paths import *

### Importing

In [2]:
%%time

OUTPUT_DIR.mkdir(exist_ok=True, parents=True)

args = ImportArguments(
    input_path=str(PARQUET_DIR),
    input_format="parquet",
    use_hipscat_index=True,
    add_hipscat_index=False,
    id_column=HIPSCAT_ID_COLUMN,
    output_catalog_name="sfd",
    output_path=OUTPUT_DIR,
    pixel_threshold=1 << 20,
    highest_healpix_order=5,
    dask_n_workers=12,
    progress_bar=True,
    overwrite=True,
    resume=False,
)
runner.pipeline(args)

Finishing : 100%|██████████| 6/6 [00:01<00:00,  5.01it/s]


CPU times: user 51.7 s, sys: 13.7 s, total: 1min 5s
Wall time: 6min 29s


### Validating

In [20]:
import numpy as np
import pandas as pd
from hipscat.pixel_math.hipscat_id import hipscat_id_to_healpix

df = pd.read_parquet('data/output/sfd/Norder=4/Dir=0/Npix=333.parquet')
df.set_index(HIPSCAT_ID_COLUMN, inplace=True)
display(df)

offsets = np.diff(df.index.values)
np.testing.assert_array_equal(
    hipscat_id_to_healpix(df.index, target_order=df['pixel_Norder']),
    df['pixel_Npix'],
)

,pixel_Norder,pixel_Npix,ebv,Norder,Dir,Npix
_hipscat_index,,,,,,
1499698675914375168,14,349175808,0.021667,4,0,333
1499698680209342464,14,349175809,0.021686,4,0,333
1499698684504309760,14,349175810,0.021684,4,0,333
1499698688799277056,14,349175811,0.021698,4,0,333
1499698693094244352,14,349175812,0.021702,4,0,333
...,...,...,...,...,...,...
1504202254066909184,14,350224379,0.016422,4,0,333
1504202258361876480,14,350224380,0.016418,4,0,333
1504202262656843776,14,350224381,0.016389,4,0,333
